In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import os.path as path
import itertools
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input,InputLayer, Dense, Activation, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler, \
                                        EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical, multi_gpu_model, Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [4]:
data_dir = 'data/'

In [ ]:
# train_data = np.load(path.join(data_dir, 'imagenet_6_class_172_train_data.npz'))
# val_data = np.load(path.join(data_dir, 'imagenet_6_class_172_val_data.npz'))

x_train = np.load(path.join(data_dir, 'imagenet_6_class_172_x_train.npy'))
y_train = np.load(path.join(data_dir, 'imagenet_6_class_172_y_train.npy'))
x_val = np.load(path.join(data_dir, 'imagenet_6_class_172_x_val.npy'))
y_val = np.load(path.join(data_dir, 'imagenet_6_class_172_y_val.npy'))
y_list = np.load(path.join(data_dir, 'imagenet_6_class_172_y_list.npy'))

In [ ]:
# x_train = train_data['x_data']
# y_train = train_data['y_data']
# x_val = val_data['x_data']
# y_val = val_data['y_data']
x_test = x_val
y_test = y_val
# y_list = val_data['y_list']

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape, y_list.shape

((235111, 172, 172, 3),
 (235111,),
 (3183, 172, 172, 3),
 (3183,),
 (3183, 172, 172, 3),
 (3183,),
 (6,))

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = y_val

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((235111, 172, 172, 3),
 (235111, 6),
 (3183, 172, 172, 3),
 (3183, 6),
 (3183, 172, 172, 3),
 (3183, 6))

In [ ]:
input_shape = x_train[0].shape
output_size = len(y_list)

In [ ]:
def build_2d_cnn_custom_ch_64_DO(conv_num=1):
    input_layer = Input(shape=input_shape)
    x = input_layer

    for i in range(conv_num):
        x = Conv2D(kernel_size=5, filters=64*(2**(i//2)), strides=(1,1), padding='same')(x)
#         x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D(pool_size=3, strides=(2,2), padding='same')(x)

    x = Flatten()(x)
  
    x = Dropout(0.5)(x)
    output_layer = Dense(output_size, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model

In [ ]:
for i in range(1, 8):
    model = build_2d_cnn_custom_ch_64_DO(conv_num=i)
    model.summary()
    del model

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 172, 172, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 172, 172, 64)      4864      
_________________________________________________________________
activation (Activation)      (None, 172, 172, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 86, 86, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 473344)            0         
_________________________________________________________________
dropout (Dropout)    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 172, 172, 3)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 172, 172, 64)      4864      
_________________________________________________________________
activation_15 (Activation)   (None, 172, 172, 64)      0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 86, 86, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 86, 86, 64)        102464    
_________________________________________________________________
activation_16 (Activation)   (None, 86, 86, 64)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 43, 43, 64)        0         
__________

In [ ]:
class BalanceDataGenerator(Sequence):
    def __init__(self, x_data, y_data, batch_size, shuffle=True):
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.sample_size = int(np.sum(y_data, axis=0).min())
        self.data_shape = x_data.shape[1:]
        self.y_label = self.y_data.argmax(axis=1)
        self.labels = np.unique(self.y_label)
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.labels) * self.sample_size / self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.zeros((len(self.labels), self.sample_size))
        for i, label in enumerate(self.labels):
            y_index = np.argwhere(self.y_label==label).squeeze()
            if self.shuffle == True:
                self.indexes[i] = np.random.choice(y_index, 
                                   self.sample_size, 
                                   replace=False)
            else:
                self.indexes[i] = y_index[:self.sample_size]
                
        self.indexes = self.indexes.flatten().astype(np.int32)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
                
    def __getitem__(self, batch_idx):
        indices = self.indexes[batch_idx*self.batch_size: (batch_idx+1)*self.batch_size]
        return self.x_data[indices], self.y_data[indices]    

In [ ]:
batch_size = 40
data_generator = BalanceDataGenerator(x_train, y_train,
                                      batch_size=batch_size)

In [ ]:
for i in range(3, 8):
    base = 'vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_050_DO'
    model_name = base+'_{}_conv'.format(i)
    model = build_2d_cnn_custom_ch_64_DO(conv_num=i)
#     model = multi_gpu_model(model, gpus=2)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4),
          metrics=['accuracy'])
    model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
    os.makedirs(model_path, exist_ok=True)
    model_filename = model_path+'{epoch:03d}-{val_loss:.4f}.hdf5'
    checkpointer = ModelCheckpoint(filepath = model_filename, monitor = "val_loss", 
                                   verbose=1, save_best_only=True)

    early_stopping = EarlyStopping(monitor='val_loss', patience=100)
    hist = model.fit_generator(data_generator,
            steps_per_epoch=len(x_train)//batch_size,
            epochs=10000,
            validation_data=(x_val, y_val),
            callbacks = [checkpointer, early_stopping],
            workers=8, 
            use_multiprocessing=True
    )

    print()
    print(model_name, 'Model')
    fig, ax = plt.subplots()
    ax.plot(hist.history['loss'], 'y', label='train loss')
    ax.plot(hist.history['val_loss'], 'r', label='val loss')
    ax.plot(hist.history['acc'], 'b', label='train acc')
    ax.plot(hist.history['val_acc'], 'g', label='val acc')
    ax.set_xlabel('epoch')
    ax.set_ylabel('loss')
    ax.legend(loc='upper left')
    plt.show()

    png_path = 'visualization/learning_curve/'
    filename = model_name+'.png'
    os.makedirs(png_path, exist_ok=True)
    fig.savefig(png_path+filename, transparent=True)

    model.save(model_path+'000_last.hdf5')
    del(model)

    model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
    model_filename = model_path + sorted(os.listdir(model_path))[-1]
    model = load_model(model_filename)
    [loss, accuracy] = model.evaluate(x_test, y_test)
    print('Loss:', loss, 'Accuracy:', accuracy)
    print()

    del(model)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10000
3200/3183 [==============================] - 53s 17ms/sample - loss: 1.1170 - acc: 0.6275

Epoch 00001: val_loss improved from inf to 1.11702, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_050_DO_3_conv_checkpoint/001-1.1170.hdf5
242/242 [==============================] - 88s 362ms/step - loss: 1.3866 - acc: 0.4306 - val_loss: 1.1170 - val_acc: 0.6275
Epoch 2/10000
3200/3183 [==============================] - 57s 18ms/sample - loss: 1.3218 - acc: 0.4375

Epoch 00002: val_loss did not improve from 1.11702
242/242 [==============================] - 89s 368ms/step - loss: 1.0847 - acc: 0.5899 - val_loss: 1.3218 - val_acc: 0.4375
Epoch 3/10000
3200/3183 [==============================] - 53s 17ms/sample - loss: 1.4405 - acc: 0.3975

Epoch 00003: val_loss did not improve from 1.11702
242/242 [==============================] - 88s 365ms/step - loss: 0.9995 - acc: 0.6167 - val_loss: 1.440

3200/3183 [==============================] - 47s 15ms/sample - loss: 0.7012 - acc: 0.7350

Epoch 00014: val_loss did not improve from 0.52942
242/242 [==============================] - 78s 321ms/step - loss: 0.5707 - acc: 0.7903 - val_loss: 0.7012 - val_acc: 0.7350
Epoch 15/10000
3200/3183 [==============================] - 45s 14ms/sample - loss: 0.4684 - acc: 0.8478

Epoch 00015: val_loss improved from 0.52942 to 0.46844, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_050_DO_3_conv_checkpoint/015-0.4684.hdf5
242/242 [==============================] - 88s 362ms/step - loss: 0.5560 - acc: 0.7911 - val_loss: 0.4684 - val_acc: 0.8478
Epoch 16/10000
3200/3183 [==============================] - 44s 14ms/sample - loss: 0.7858 - acc: 0.7072

Epoch 00016: val_loss did not improve from 0.46844
242/242 [==============================] - 270s 1s/step - loss: 0.5488 - acc: 0.7957 - val_loss: 0.7858 - val_acc: 0.7072
Epoch 17/10000
3200/3183 [============

3200/3183 [==============================] - 46s 14ms/sample - loss: 0.5319 - acc: 0.8253

Epoch 00029: val_loss did not improve from 0.44440
242/242 [==============================] - 81s 337ms/step - loss: 0.4166 - acc: 0.8445 - val_loss: 0.5319 - val_acc: 0.8253
Epoch 30/10000
3200/3183 [==============================] - 40s 13ms/sample - loss: 0.6393 - acc: 0.7806

Epoch 00030: val_loss did not improve from 0.44440
242/242 [==============================] - 80s 331ms/step - loss: 0.4085 - acc: 0.8484 - val_loss: 0.6393 - val_acc: 0.7806
Epoch 31/10000
3200/3183 [==============================] - 45s 14ms/sample - loss: 0.4389 - acc: 0.8556

Epoch 00031: val_loss improved from 0.44440 to 0.43894, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_050_DO_3_conv_checkpoint/031-0.4389.hdf5
242/242 [==============================] - 78s 323ms/step - loss: 0.4039 - acc: 0.8516 - val_loss: 0.4389 - val_acc: 0.8556
Epoch 32/10000
3200/3183 [==========

3200/3183 [==============================] - 81s 25ms/sample - loss: 0.4933 - acc: 0.8234

Epoch 00044: val_loss did not improve from 0.35115
242/242 [==============================] - 112s 463ms/step - loss: 0.3419 - acc: 0.8776 - val_loss: 0.4933 - val_acc: 0.8234
Epoch 45/10000
3200/3183 [==============================] - 55s 17ms/sample - loss: 0.3618 - acc: 0.8675

Epoch 00045: val_loss did not improve from 0.35115
242/242 [==============================] - 85s 351ms/step - loss: 0.3313 - acc: 0.8775 - val_loss: 0.3618 - val_acc: 0.8675
Epoch 46/10000
3200/3183 [==============================] - 95s 30ms/sample - loss: 0.6503 - acc: 0.7672

Epoch 00046: val_loss did not improve from 0.35115
242/242 [==============================] - 133s 548ms/step - loss: 0.3227 - acc: 0.8806 - val_loss: 0.6503 - val_acc: 0.7672
Epoch 47/10000
3200/3183 [==============================] - 81s 25ms/sample - loss: 0.5607 - acc: 0.8078

Epoch 00047: val_loss did not improve from 0.35115
242/242 [====

Epoch 59/10000
3200/3183 [==============================] - 129s 40ms/sample - loss: 0.3717 - acc: 0.8725

Epoch 00059: val_loss did not improve from 0.24916
242/242 [==============================] - 162s 669ms/step - loss: 0.2988 - acc: 0.8894 - val_loss: 0.3717 - val_acc: 0.8725
Epoch 60/10000
3200/3183 [==============================] - 66s 21ms/sample - loss: 0.4062 - acc: 0.8700

Epoch 00060: val_loss did not improve from 0.24916
242/242 [==============================] - 96s 397ms/step - loss: 0.2959 - acc: 0.8949 - val_loss: 0.4062 - val_acc: 0.8700
Epoch 61/10000
3200/3183 [==============================] - 149s 47ms/sample - loss: 0.2887 - acc: 0.8959

Epoch 00061: val_loss did not improve from 0.24916
242/242 [==============================] - 197s 815ms/step - loss: 0.2885 - acc: 0.8932 - val_loss: 0.2887 - val_acc: 0.8959
Epoch 62/10000
3200/3183 [==============================] - 136s 42ms/sample - loss: 0.2685 - acc: 0.9159

Epoch 00062: val_loss did not improve from 0.2

3200/3183 [==============================] - 63s 20ms/sample - loss: 0.3392 - acc: 0.8963

Epoch 00074: val_loss did not improve from 0.24916
242/242 [==============================] - 102s 422ms/step - loss: 0.2510 - acc: 0.9050 - val_loss: 0.3392 - val_acc: 0.8963
Epoch 75/10000
3200/3183 [==============================] - 81s 25ms/sample - loss: 0.4440 - acc: 0.8675

Epoch 00075: val_loss did not improve from 0.24916
242/242 [==============================] - 163s 672ms/step - loss: 0.2551 - acc: 0.9046 - val_loss: 0.4440 - val_acc: 0.8675
Epoch 76/10000
3200/3183 [==============================] - 107s 33ms/sample - loss: 0.3315 - acc: 0.8925

Epoch 00076: val_loss did not improve from 0.24916
242/242 [==============================] - 153s 632ms/step - loss: 0.2584 - acc: 0.9050 - val_loss: 0.3315 - val_acc: 0.8925
Epoch 77/10000
3200/3183 [==============================] - 87s 27ms/sample - loss: 0.3514 - acc: 0.8925

Epoch 00077: val_loss did not improve from 0.24916
242/242 [==

3200/3183 [==============================] - 94s 29ms/sample - loss: 0.3543 - acc: 0.8775

Epoch 00089: val_loss did not improve from 0.24179
242/242 [==============================] - 124s 514ms/step - loss: 0.2281 - acc: 0.9178 - val_loss: 0.3543 - val_acc: 0.8775
Epoch 90/10000
3200/3183 [==============================] - 46s 14ms/sample - loss: 0.4174 - acc: 0.8653

Epoch 00090: val_loss did not improve from 0.24179
242/242 [==============================] - 102s 422ms/step - loss: 0.2364 - acc: 0.9120 - val_loss: 0.4174 - val_acc: 0.8653
Epoch 91/10000
3200/3183 [==============================] - 136s 42ms/sample - loss: 0.3039 - acc: 0.9150

Epoch 00091: val_loss did not improve from 0.24179
242/242 [==============================] - 174s 719ms/step - loss: 0.2219 - acc: 0.9205 - val_loss: 0.3039 - val_acc: 0.9150
Epoch 92/10000
3200/3183 [==============================] - 96s 30ms/sample - loss: 0.3744 - acc: 0.8725

Epoch 00092: val_loss did not improve from 0.24179
242/242 [==

Epoch 105/10000
3200/3183 [==============================] - 95s 30ms/sample - loss: 0.4216 - acc: 0.8666

Epoch 00105: val_loss did not improve from 0.24179
242/242 [==============================] - 135s 556ms/step - loss: 0.2108 - acc: 0.9256 - val_loss: 0.4216 - val_acc: 0.8666
Epoch 106/10000
3200/3183 [==============================] - 123s 38ms/sample - loss: 0.3170 - acc: 0.8825

Epoch 00106: val_loss did not improve from 0.24179
242/242 [==============================] - 157s 647ms/step - loss: 0.2151 - acc: 0.9251 - val_loss: 0.3170 - val_acc: 0.8825
Epoch 107/10000
3200/3183 [==============================] - 117s 37ms/sample - loss: 0.2965 - acc: 0.9078

Epoch 00107: val_loss did not improve from 0.24179
242/242 [==============================] - 149s 615ms/step - loss: 0.2134 - acc: 0.9231 - val_loss: 0.2965 - val_acc: 0.9078
Epoch 108/10000
3200/3183 [==============================] - 88s 27ms/sample - loss: 0.3215 - acc: 0.9000

Epoch 00108: val_loss did not improve from

In [ ]:
log_dir = 'log'
os.makedirs(log_dir, exist_ok=True)
base = 'vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_050_DO'

with open(path.join(log_dir, base), 'w') as log_file:
    for i in range(3, 8):
        model_name = base+'_{}_conv'.format(i)
        print()
        print(model_name, 'Model')
        model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
        model_filename = model_path + sorted(os.listdir(model_path))[-1]

        model = load_model(model_filename)
        model.summary()

        [loss, accuracy] = model.evaluate(x_test, y_test)
        print('Loss:', loss, 'Accuracy:', accuracy)

        del(model)

        log_file.write('\t'.join([model_name, str(accuracy), str(loss)])+'\n')

In [ ]:
for i in range(3, 8):
    model_name = base+'_{}_conv'.format(i)
    print()
    print(model_name, 'Model')
    model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
    model_filename = model_path + '000_last.hdf5'

    model = load_model(model_filename)
    model.summary()

    [loss, accuracy] = model.evaluate(x_test, y_test)
    print('Loss:', loss, 'Accuracy:', accuracy)

    del(model)